In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
from sklearn.model_selection import train_test_split
from sktime.forecasting.arima import AutoARIMA

In [4]:
data = pd.read_csv('parkingLot.csv')
data

,vehicle_no,timestamp,camera_id
0,MHUN7063,2024-09-12 05:00:00,1
1,MHYN4677,2024-09-12 05:00:00,1
2,MHEL6595,2024-09-12 05:00:00,1
3,MHNQ2590,2024-09-12 05:00:00,1
4,MHHA0518,2024-09-12 05:00:00,1
...,...,...,...
106689,MHCN7114,2024-11-14 00:00:00,2
106690,MHVY8418,2024-11-14 00:00:00,2
106691,MHSR8102,2024-11-14 00:00:00,2
106692,MHQG5264,2024-11-14 00:00:00,2


In [5]:
# data['vehicle_no'].fillna('Unknown', inplace=True)
data['time'] = pd.to_datetime(data['timestamp'])
# data['date'] = pd.to_datetime(data['timestamp'])
data['date'] = data['time'].dt.date
# if time stamp time is 00 00 then make it's date as previous date
data.loc[data['time'].dt.time == datetime.time(0, 0), 'date'] = data['date'] - pd.Timedelta(days=1)
data

,vehicle_no,timestamp,camera_id,time,date
0,MHUN7063,2024-09-12 05:00:00,1,2024-09-12 05:00:00,2024-09-12
1,MHYN4677,2024-09-12 05:00:00,1,2024-09-12 05:00:00,2024-09-12
2,MHEL6595,2024-09-12 05:00:00,1,2024-09-12 05:00:00,2024-09-12
3,MHNQ2590,2024-09-12 05:00:00,1,2024-09-12 05:00:00,2024-09-12
4,MHHA0518,2024-09-12 05:00:00,1,2024-09-12 05:00:00,2024-09-12
...,...,...,...,...,...
106689,MHCN7114,2024-11-14 00:00:00,2,2024-11-14 00:00:00,2024-11-13
106690,MHVY8418,2024-11-14 00:00:00,2,2024-11-14 00:00:00,2024-11-13
106691,MHSR8102,2024-11-14 00:00:00,2,2024-11-14 00:00:00,2024-11-13
106692,MHQG5264,2024-11-14 00:00:00,2,2024-11-14 00:00:00,2024-11-13


In [6]:
data['camera_1'] = data['camera_id'] == 1
data['camera_2'] = data['camera_id'] == 2
data['camera_1'] = 1 * data['camera_1']
data['camera_2'] = 1 * data['camera_2']

In [7]:
def mean_absolute_scaled_error(y_true, y_pred):
    """
    Calculate the Mean Absolute Scaled Error (MASE)
    
    y_true - array-like of true values
    y_pred - array-like of predicted values
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mae = np.mean(np.abs(y_true - y_pred))  # Mean Absolute Error (MAE)
    naive_mae = np.mean(np.abs(y_true[1:] - y_true[:-1]))  # Mean Absolute Error of naive forecast
    return mae / naive_mae

In [8]:
def mean_absolute_error(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.mean(np.abs(y_true - y_pred))

In [9]:
def mean_absolute_percentage_error(y_true, y_pred):
    """
    Calculate the Mean Absolute Percentage Error (MAPE)
    
    y_true - array-like of true values
    y_pred - array-like of predicted values
    """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [13]:
na_data = data[~ data['vehicle_no'].isna()]
na_data.groupby(['date', 'time']).size()
na_dates = na_data['date'].unique()
data[data['date'].isin(na_dates)]

,vehicle_no,timestamp,camera_id,time,date,camera_1,camera_2
0,MHUN7063,2024-09-12 05:00:00,1,2024-09-12 05:00:00,2024-09-12,1,0
1,MHYN4677,2024-09-12 05:00:00,1,2024-09-12 05:00:00,2024-09-12,1,0
2,MHEL6595,2024-09-12 05:00:00,1,2024-09-12 05:00:00,2024-09-12,1,0
3,MHNQ2590,2024-09-12 05:00:00,1,2024-09-12 05:00:00,2024-09-12,1,0
4,MHHA0518,2024-09-12 05:00:00,1,2024-09-12 05:00:00,2024-09-12,1,0
...,...,...,...,...,...,...,...
106689,MHCN7114,2024-11-14 00:00:00,2,2024-11-14 00:00:00,2024-11-13,0,1
106690,MHVY8418,2024-11-14 00:00:00,2,2024-11-14 00:00:00,2024-11-13,0,1
106691,MHSR8102,2024-11-14 00:00:00,2,2024-11-14 00:00:00,2024-11-13,0,1
106692,MHQG5264,2024-11-14 00:00:00,2,2024-11-14 00:00:00,2024-11-13,0,1


In [14]:
car_data = pd.DataFrame(columns=['date', 'arrival_time', 'exit_time'])
from tqdm import tqdm
for entry in tqdm(na_data.iterrows()):
    entry = entry[1]
    # print(entry, type(entry))
    vehicle = entry['vehicle_no']
    date = entry['date']
    time = entry['time']
    camera = entry['camera_id']
    vehicle = entry['vehicle_no']
    
    if vehicle in car_data.index:
        if camera == 2:  # Assuming camera 2 is for exit
            car_data.at[vehicle, 'exit_time'] = time
        if camera == 1:
            car_data.at[vehicle, 'arrival_time'] = time
    else:
        if camera == 1:  # Assuming camera 1 is for arrival
            car_data.loc[vehicle] = [date, time, None]
        if camera == 2:
            car_data.loc[vehicle] = [date, None, time]

0it [00:00, ?it/s]

234it [00:00, 606.62it/s]/tmp/ipykernel_323942/1936623831.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  car_data.loc[vehicle] = [date, None, time]
373it [00:00, 874.52it/s]/tmp/ipykernel_323942/1936623831.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  car_data.loc[vehicle] = [date, None, time]
786it [00:00, 923.83it/s] /tmp/ipykernel_323942/1936623831.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this w

In [15]:
# car_data.to_csv('car_data.csv', index=True) 

In [16]:
car_data = pd.read_csv('car_data.csv')
car_data

,vehicle_no,date,arrival_time,exit_time
0,MHUN7063,2024-09-12,2024-09-12 05:00:00,2024-09-12 08:00:00
1,MHYN4677,2024-09-12,2024-09-12 05:00:00,2024-09-12 08:00:00
2,MHEL6595,2024-09-12,2024-09-12 05:00:00,2024-09-12 08:00:00
3,MHNQ2590,2024-09-12,2024-09-12 05:00:00,2024-09-12 08:20:00
4,MHHA0518,2024-09-12,2024-09-12 05:00:00,NaN
...,...,...,...,...
53671,MHQJ4009,2024-11-13,2024-11-13 20:00:00,2024-11-14 00:00:00
53672,MHPN3450,2024-11-13,2024-11-13 20:00:00,2024-11-14 00:00:00
53673,MHBB9719,2024-11-13,2024-11-13 20:00:00,2024-11-14 00:00:00
53674,MHLE56X9,2024-11-13,NaN,2024-11-13 22:40:00


In [17]:
grouped_car_data = [group for _, group in car_data.groupby('date')]
print(grouped_car_data)

[    vehicle_no        date         arrival_time            exit_time
0     MHUN7063  2024-09-12  2024-09-12 05:00:00  2024-09-12 08:00:00
1     MHYN4677  2024-09-12  2024-09-12 05:00:00  2024-09-12 08:00:00
2     MHEL6595  2024-09-12  2024-09-12 05:00:00  2024-09-12 08:00:00
3     MHNQ2590  2024-09-12  2024-09-12 05:00:00  2024-09-12 08:20:00
4     MHHA0518  2024-09-12  2024-09-12 05:00:00                  NaN
..         ...         ...                  ...                  ...
894   MHOQ0040  2024-09-12  2024-09-12 20:20:00  2024-09-13 00:00:00
895   MHQK9295  2024-09-12  2024-09-12 20:20:00  2024-09-12 23:20:00
896   MHIA1330  2024-09-12  2024-09-12 20:40:00  2024-09-13 00:00:00
897   MHHM2699  2024-09-12                  NaN  2024-09-12 22:00:00
898   MHR91492  2024-09-12                  NaN  2024-09-12 23:00:00

[899 rows x 4 columns],      vehicle_no        date         arrival_time            exit_time
899    MHTR7927  2024-09-13  2024-09-13 05:00:00  2024-09-13 10:40:00
900   

In [19]:
# hamming distance
def hamming_distance(s1, s2):
    return sum(c1 != c2 for c1, c2 in zip(s1, s2))

# check if hamming distance is 1
def check_hamming_distance(s1, s2):
    return hamming_distance(s1, s2) == 1

      vehicle_no        date         arrival_time            exit_time
0       MHUN7063  2024-09-12  2024-09-12 05:00:00  2024-09-12 08:00:00
1       MHYN4677  2024-09-12  2024-09-12 05:00:00  2024-09-12 08:00:00
2       MHEL6595  2024-09-12  2024-09-12 05:00:00  2024-09-12 08:00:00
3       MHNQ2590  2024-09-12  2024-09-12 05:00:00  2024-09-12 08:20:00
4       MHHA0518  2024-09-12  2024-09-12 05:00:00                  NaN
...          ...         ...                  ...                  ...
53671   MHQJ4009  2024-11-13  2024-11-13 20:00:00  2024-11-14 00:00:00
53672   MHPN3450  2024-11-13  2024-11-13 20:00:00  2024-11-14 00:00:00
53673   MHBB9719  2024-11-13  2024-11-13 20:00:00  2024-11-14 00:00:00
53674   MHLE56X9  2024-11-13                  NaN  2024-11-13 22:40:00
53675   MHDG83S5  2024-11-13                  NaN  2024-11-13 22:40:00

[53676 rows x 4 columns]


/tmp/ipykernel_323942/3643548316.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incomplete_data.drop(idx2, inplace=True)
/tmp/ipykernel_323942/3643548316.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incomplete_data.drop(idx2, inplace=True)
/tmp/ipykernel_323942/3643548316.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incomplete_data.drop(idx2, inplace=True)
/tmp/ipykernel_323942/3643548316.py:14: SettingWithCopyWar

In [22]:
combined_car_data = pd.DataFrame(columns=car_data.columns)
print(car_data)
for group in grouped_car_data:
    incomplete_data = group[group['exit_time'].isna() | group['arrival_time'].isna()]
    complete_data = group.dropna(subset=['exit_time', 'arrival_time'])
    # print(complete_data)
    combined_car_data = pd.concat([combined_car_data, complete_data], ignore_index=True)
    for idx, row in incomplete_data.iterrows():
        if row['exit_time'] is np.nan:
            for idx2, row2 in incomplete_data.iterrows():
                if row2['exit_time'] is not np.nan:
                    if check_hamming_distance(row['vehicle_no'], row2['vehicle_no']):
                        row['exit_time'] = row2['exit_time']
                        incomplete_data.drop(idx2, inplace=True)
                        break
    incomplete_data = incomplete_data.dropna(subset=['exit_time','arrival_time'])   
    combined_car_data = pd.concat([combined_car_data, incomplete_data], ignore_index=True)

    # print(incomplete_data)
    # combined_car_data = pd.concat([combined_car_data, incomplete_data], ignore_index=True)

# Remove any remaining rows with NaN in 'arrival_time' or 'exit_time'
combined_car_data.dropna(subset='arrival_time', inplace=True)
combined_car_data.dropna(subset=['exit_time', 'arrival_time'], inplace=True)
combined_car_data.sort_values(by='arrival_time', inplace=True)

      vehicle_no        date         arrival_time            exit_time
0       MHUN7063  2024-09-12  2024-09-12 05:00:00  2024-09-12 08:00:00
1       MHYN4677  2024-09-12  2024-09-12 05:00:00  2024-09-12 08:00:00
2       MHEL6595  2024-09-12  2024-09-12 05:00:00  2024-09-12 08:00:00
3       MHNQ2590  2024-09-12  2024-09-12 05:00:00  2024-09-12 08:20:00
4       MHHA0518  2024-09-12  2024-09-12 05:00:00                  NaN
...          ...         ...                  ...                  ...
53671   MHQJ4009  2024-11-13  2024-11-13 20:00:00  2024-11-14 00:00:00
53672   MHPN3450  2024-11-13  2024-11-13 20:00:00  2024-11-14 00:00:00
53673   MHBB9719  2024-11-13  2024-11-13 20:00:00  2024-11-14 00:00:00
53674   MHLE56X9  2024-11-13                  NaN  2024-11-13 22:40:00
53675   MHDG83S5  2024-11-13                  NaN  2024-11-13 22:40:00

[53676 rows x 4 columns]


/tmp/ipykernel_323942/3643548316.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incomplete_data.drop(idx2, inplace=True)
/tmp/ipykernel_323942/3643548316.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incomplete_data.drop(idx2, inplace=True)
/tmp/ipykernel_323942/3643548316.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incomplete_data.drop(idx2, inplace=True)
/tmp/ipykernel_323942/3643548316.py:14: SettingWithCopyWar

In [20]:
# Convert arrival_time and exit_time to datetime if they are not already
combined_car_data['arrival_time'] = pd.to_datetime(combined_car_data['arrival_time'])
combined_car_data['exit_time'] = pd.to_datetime(combined_car_data['exit_time'])

# Calculate the duration
combined_car_data['duration'] = combined_car_data['exit_time'] - combined_car_data['arrival_time']

# Display the updated DataFrame
combined_car_data.sort_values(by='date', inplace=True)
print(combined_car_data)

      vehicle_no        date        arrival_time           exit_time  \
0       MHUN7063  2024-09-12 2024-09-12 05:00:00 2024-09-12 08:00:00   
565     MHXW1211  2024-09-12 2024-09-12 14:20:00 2024-09-12 19:20:00   
566     MHIG3726  2024-09-12 2024-09-12 14:20:00 2024-09-12 17:20:00   
578     MHBV7539  2024-09-12 2024-09-12 14:20:00 2024-09-12 17:20:00   
568     MHAH3130  2024-09-12 2024-09-12 14:20:00 2024-09-12 18:40:00   
...          ...         ...                 ...                 ...   
51768   MHAJ0127  2024-11-13 2024-11-13 09:20:00 2024-11-13 15:00:00   
51765   MHZQ2674  2024-11-13 2024-11-13 09:20:00 2024-11-13 15:20:00   
51766   MHUJ4069  2024-11-13 2024-11-13 09:20:00 2024-11-13 15:40:00   
51711   MHQN8905  2024-11-13 2024-11-13 08:20:00 2024-11-13 12:20:00   
52275   MHBB9719  2024-11-13 2024-11-13 20:00:00 2024-11-14 00:00:00   

             duration  
0     0 days 03:00:00  
565   0 days 05:00:00  
566   0 days 03:00:00  
578   0 days 03:00:00  
568   0 days 04

In [23]:
grouped_duration_by_date = combined_car_data.groupby('date')['duration'].mean()
print(grouped_duration_by_date)
train_duration = grouped_duration_by_date[:56]
test_duration = grouped_duration_by_date[56:]

KeyError: 'Column not found: duration'

In [24]:
from pmdarima import auto_arima
auto_model_duration = auto_arima(train_duration,
                        # start_p=0, max_p=14,    # Range of values for p
                        # start_q=0, max_q=14,    # Range of values for q
                        p = None,               # Let auto_arima determine the best p
                        q = None,               # Let auto_arima determine the best q
                        d = 0,                # Let auto_arima determine the best d
                        seasonal=True,         # Enable seasonal differencing
                        m=7,                   # Weekly seasonality
                        # start_P=0, max_P=14,    # Range of values for seasonal P
                        # start_Q=0, max_Q=14,    # Range of values for seasonal Q
                        P=None,                # Let auto_arima determine the best P
                        Q=None,                # Let auto_arima determine the best Q

                        D=None,                # Let auto_arima determine the best D
                        trace=True,            # Print results for each model tested
                        # error_action='ignore', # Ignore models that don’t fit
                        suppress_warnings=True,
                        stepwise=True)         # Stepwise search for faster execution

# Display the best model found
print(auto_model_duration.summary())

NameError: name 'train_duration' is not defined

In [ ]:
# Forecast the test set
forecast_duration = auto_model_duration.predict(n_periods=len(test_duration))
plt.plot(test_duration.index, test_duration, label='actual')
plt.plot(test_duration.index, forecast_duration, label='forecast')
plt.legend()
plt.show()

In [ ]:
forecast_duration = pd.to_timedelta(forecast_duration)

plt.plot(test_duration.index, test_duration, label='actual')
plt.plot(test_duration.index, forecast_duration, label='forecast')
plt.legend()
plt.show()

In [ ]:

forecast_duration = pd.to_timedelta(forecast_duration)
print(forecast_duration)
print(test_duration)


mae_duration = mean_absolute_error(test_duration, forecast_duration)
mape_duration = mean_absolute_percentage_error(test_duration, forecast_duration)
mae_duration = pd.to_timedelta(mae_duration)
print(type(mae_duration))
print(f'Mean Absolute Error: {mae_duration}')
print(f'Mean Absolute Percentage Error: {mape_duration}')